

| column name    | meaning                 |                                                                                                                         
| :-------------- | :----------------------- |                                                        
| family_fot     | 英文科名                |  
| family_fot_zh  | 中文科名                |     
| family         | Family (APG IV & PPG I) |     
| family_zh      | 科名 (APG IV & PPG I)   |      
| fullname       | 學名                    |       
| zh_name        | 中文名                  |       
| prev_category  | 2008-2010 Red List      |     
| prev_criteria  | 評估標準                |       
| prev_evaluator | 評估者                  |      
| category       | 2017 Red List           |    
| criteria       | 2017評估標準            |    
| evaluator      | 2017評估者              |      
| evaluation     | 2017評估說明書          |   
| images         | 2017圖片                |    
| endemic        | 特有性                  |      
| native         | 原生性                  |      
| others         | 其他                    |      
| note           | 備註                    |     

In [1]:
import psycopg2 as pg
import genlist_api
import re
import csv
g = genlist_api.Genlist()
# postgresql database name
DSN = 'dbname=nvdimp user=psilotum'
conn = pg.connect(DSN)

## Create data table
twredlist2017

In [ ]:
#id|family_fot|family_fot_zh|family|family_zh|fullname|
#zh_name|prev_category|prev_criteria|prev_evaluator|
#category|criteria|evaluator|evaluation|images|endemic|native|
#others|note|update

CREATE_TABLE = '''
CREATE TABLE nomenclature.twredlist2017_backup AS (SELECT * FROM nomenclature.twredlist2017);
DROP TABLE IF EXISTS nomenclature.twredlist2017;                                                                                                                                   
CREATE TABLE nomenclature.twredlist2017 (                                                                                                                                          
    id serial primary key,                                                                                                                                            
    plant_type integer,                                                                                                                                               
    family_fot varchar,                                                                                                                                               
    family_fot_zh varchar,                                                                                                                                            
    family varchar,                                                                                                                                                   
    family_zh varchar,                                                                                                                                                
    genus varchar,                                                                                                                                                    
    genus_zh varchar,                                                                                                                                                 
    name varchar,                                                                                                                                                     
    fullname varchar,                                                                                                                                                 
    zh_name varchar,                                                                                                                                                  
    authors varchar,                                                                                                                                                  
    prev_category varchar,                                                                                                                                            
    prev_criteria varchar,                                                                                                                                            
    prev_evaluator varchar,                                                                                                                                           
    category varchar,                                                                                                                                                 
    criteria varchar,                                                                                                                                                 
    evaluator varchar,                                                                                                                                                
    evaluation varchar,                                                                                                                                               
    images varchar,                                                                                                                                                   
    endemic varchar,                                                                                                                                                  
    native varchar,                                                                                                                                                   
    others varchar,                                                                                                                                                   
    note varchar,
    update varchar,
    tropicos_namecode varchar,
    tropicos_fullname varchar,
    ipni_id varchar,
    ipni_fullname varchar,
    ipni_name varchar
);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)
    conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum
# 

redlistRaw = []
with open('../../workspace/redlist171211.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
r = redlistRaw

#rlistRaw = pd.read_csv('../../workspace/redlist171204.csv', delimiter='|', na_values='')

In [ ]:
for row in range(len(r)):
    # escape quote (for postgresql)
    if "'" in r[row][5]:
        r[row][5] = r[row][5].replace("'", "''")
    eachRow = "'" + "','".join(r[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' × ', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist2017 
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,prev_category,prev_criteria,prev_evaluator,                                                                                                                   
    category,criteria,evaluator,evaluation,images,endemic,                                                                                                                
    native,others,note,update) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
# 更新 f. 為 fo.
UPDATE_FORMA = '''
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, ' f. ', ' fo. ')
where criteria like  '品型%'
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_FORMA)
    conn.commit()

In [ ]:
# 更正一些奇怪的東西
UPDATE_OTHERS = '''
UPDATE nomenclature.twredlist2017
SET family = 'Typhaceae' WHERE family = 'Typhaceae ';
UPDATE nomenclature.twredlist2017
SET family = 'Typhaceae' WHERE family = 'Typhaceae ';
UPDATE nomenclature.twredlist2017
SET family = 'Tamaricaceae' WHERE family = 'Tamaricaeae';
UPDATE nomenclature.twredlist2017
SET family = 'Isoëtaceae' where family = 'Isoetaceae';
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '#', '');
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '\?', '');
UPDATE nomenclature.twredlist2017
SET fullname = regexp_replace(fullname, '\*', '');
''' 
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_OTHERS)
    conn.commit()

## 新增作者不含學名之欄位

為了方便比對，將 fullname (含作者) 拆成學名(name)以及作者(authors)

In [ ]:
QUERY = 'SELECT fullname FROM twredlist2017;'
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(QUERY)
        fullnames = curs.fetchall()
for i in range(len(fullnames)):
    fname = fullnames[i][0]
    nameFormatted = g.fmtname(fname, split=True, format_type='custom', 
                                 italic_b="", italic_e="")
    if "'" in fname:
        fname = fname.replace("'", "''")
    if "\"" in fname:
        fname = fname.replace("\"", "''")
    if "'" in nameFormatted[0]:
        nameFormatted[0] = nameFormatted[0].replace("'", "''")
    if "'" in nameFormatted[1]:
        nameFormatted[1] = nameFormatted[1].replace("'", "''")
    UPDATE = '''
        UPDATE twredlist2017 SET
            name = '%s', authors = '%s'
        WHERE fullname = '%s'
        ''' % (nameFormatted[0], nameFormatted[1], fname)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(UPDATE)
conn.commit()

### 更新屬名以及屬中名

屬中名的資料表 nomenclature.twp_genus

In [2]:
UPDATE_GENUS = '''
UPDATE nomenclature.twredlist2017
SET genus = split_part(name, ' ', 1);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_GENUS)
conn.commit()

In [3]:
UPDATE_GENUS_ZH = '''
UPDATE nomenclature.twredlist2017
SET genus_zh = g.genus_zh
FROM (SELECT * FROM nomenclature.twp_genus) as g
WHERE twredlist2017.genus = g.genus
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_GENUS_ZH)
conn.commit()

In [ ]:
# 更新 plant_type
UPDATE_PLANT_TYPE = '''
UPDATE nomenclature.twredlist2017
SET plant_type = p.plant_type
FROM (SELECT * FROM nomenclature.plant_type) as p
WHERE twredlist2017.family = p.family;
UPDATE nomenclature.twredlist2017
SET plant_type = 3 WHERE plant_type = 4;
UPDATE nomenclature.twredlist2017
SET plant_type = 0 WHERE family='Lycopodiaceae' or
family = 'Isoëtaceae' or family = 'Selaginellaceae';
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_PLANT_TYPE)
conn.commit()

In [ ]:
# 未成對的 " 會造成 zhmakeindex 爛掉
UPDATE_BAMBUSA = '''
UPDATE nomenclature.twredlist2017
SET name = 'Bambusa dolichomerithalla cv. ''Green\x0astripe-stem\'\'',
authors = 'W. C. Lin'
WHERE zh_name = '金絲火廣竹';
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(UPDATE_BAMBUSA)
conn.commit()

In [ ]:
Q = '''SELECT name from twredlist2017 where name like '%×%';'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(Q)
        qry = curs.fetchall()

In [ ]:
FULLNAME = g.fmtnameNew('Cornopteris decurrenti-alata (Hook.) Nakai var. decurrenti-alata', 
             format_type= 'custom', italic_b='\\textit{', italic_e='}',
             withSpAuthor= True)[1]

In [ ]:
g.fmtnameNew('Cornopteris decurrenti-alata (Hook.) Nakai var. decurrenti-alata')

## 2017-12-19 最後資料彙整

因為和明後來有再修正，所以需要對照哪些地方有修正過

In [ ]:
#id|family_fot|family_fot_zh|family|family_zh|fullname|
#zh_name|prev_category|prev_criteria|prev_evaluator|
#category|criteria|evaluator|evaluation|images|endemic|native|
#others|note|update

CREATE_TABLE = '''
-- CREATE TABLE nomenclature.twredlist20171219_backup AS (SELECT * FROM nomenclature.twredlist2017);
DROP TABLE IF EXISTS nomenclature.twredlist20171219;                                                                                                                                   
CREATE TABLE nomenclature.twredlist20171219 (                                                                                                                                          
    id serial primary key,                                                                                                                                            
    plant_type integer,                                                                                                                                               
    family_fot varchar,                                                                                                                                               
    family_fot_zh varchar,                                                                                                                                            
    family varchar,                                                                                                                                                   
    family_zh varchar,                                                                                                                                                
    genus varchar,                                                                                                                                                    
    genus_zh varchar,                                                                                                                                                 
    name varchar,                                                                                                                                                     
    fullname varchar,                                                                                                                                                 
    zh_name varchar,                                                                                                                                                  
    authors varchar,                                                                                                                                                  
    prev_category varchar,                                                                                                                                            
    prev_criteria varchar,                                                                                                                                            
    prev_evaluator varchar,                                                                                                                                           
    category varchar,                                                                                                                                                 
    criteria varchar,                                                                                                                                                 
    evaluator varchar,                                                                                                                                                
    evaluation varchar,                                                                                                                                               
    images varchar,                                                                                                                                                   
    endemic varchar,                                                                                                                                                  
    native varchar,                                                                                                                                                   
    others varchar,                                                                                                                                                   
    note varchar,
    update varchar,
    tropicos_namecode varchar,
    tropicos_fullname varchar,
    ipni_id varchar,
    ipni_fullname varchar,
    ipni_name varchar
);
'''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)
    conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum

redlistRaw = []
with open('../../workspace/20171219/twredlist1219.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
r = redlistRaw

for row in range(len(r)):
    # escape quote (for postgresql)
    if "'" in r[row][5]:
        r[row][5] = r[row][5].replace("'", "''")
    eachRow = "'" + "','".join(r[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' × ', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist20171219 
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,category,criteria,endemic,                                                                                                                
    native) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
# 匯入 csv 檔
# 匯入前要把斷行檢查一下
# Limnocharis flava
# Sorghum bicolor subsp. arundinaceum

redlistRaw = []
with open('../../workspace/20171219/newlyadded_1220.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
nA = redlistRaw

for row in range(len(nA)):
    # escape quote (for postgresql)
    if "'" in nA[row][5]:
        nA[row][5] = nA[row][5].replace("'", "''")
    eachRow = "'" + "','".join(nA[row]) + "'"
    # replace full-width braces
    eachRow = eachRow.replace('（', ' (')
    eachRow = eachRow.replace(' ）', ') ')
    # replace 雜交的 "x" (alphabet) to × (cross sign)
    eachRow = re.sub(' x ', ' ×', eachRow)
    # replace spaces greater than 1 
    eachRow = re.sub(" {1,}",' ', eachRow)
    # replace trailing spaces 
    eachRow = re.sub(" {1,}'","'", eachRow)
    # replace non-break spaces
    eachRow = eachRow.replace('\xa0',' ')
    # replace ssp. to subsp.
    eachRow = re.sub(" ssp. ", " subsp. ", eachRow)
    
    INSERT_REDLIST = '''
    INSERT INTO nomenclature.twredlist2017
    (id,family_fot,family_fot_zh,family,family_zh,fullname,                                                                                                                   
    zh_name,prev_category,prev_criteria,category,criteria,endemic,                                                                                                                
    native) VALUES (%s)
    ''' % (eachRow)
    with conn:
        with conn.cursor() as curs:                                                                                                                           
            curs.execute(INSERT_REDLIST)
        conn.commit()

In [ ]:
fullname = 'Oxalis acetosella L. subsp. griffithii (Edgew. & Hook f.) H.Hara var. formosana S.F.Huang & T.C.Huang'

In [ ]:
g.fmtnameNew(iname, format_type= 'custom', italic_b='\\textit{', italic_e='}')

# 1. 四名

In [ ]:
epithetLst = []                                                                                                                                
epithet = ''                                                                                                                                   
author_start = ''                                                                                                                              
cross_type = ''                                                                                                                                
subordinate_status = []
fullname_split = fullname.split(' ')
subrank = ['subsp.', 'ssp.', 'var.', 'fo.', 'cv.', '×', 'x']
for s in range(0,len(subrank)):                                                                                                                
      if subrank[s] in fullname_split:                                                                                                           
          subrank_idx = fullname_split.index(subrank[s])                                                                                         
          if subrank[s] == '×' or subrank[s] == 'x':                                                                                             
              # 把 x 取代成 ×                                                                                                                    
              subrank[s] = '×'                                                                                                                   
              fullname_split[subrank_idx] = '×'                                                                                                  
              # × 後面有跟著屬名的                                                                                                               
              # ex: Genus epithet × Genus epithet2                                                                                               
              if re.search('[A-Z].*', fullname_split[subrank_idx+1]):                                                                            
                  subordinate_status.append([subrank_idx, subrank[s]])                                                                           
                  cross_type = 1                                                                                                                 
              else:                                                                                                                              
                  # 沒有屬名的 case                                                                                                              
                  fullname_split[subrank_idx+1] = '×' + fullname_split[subrank_idx+1]                                                            
                  fullname_split.remove(subrank[s])                                                                                              
          else:                                                                                                                                  
              subordinate_status.append([subrank_idx, subrank[s]])                                                                               

          # check for autonyms                                                                                                                   
          epithet = fullname_split[1]                                                                                                            
          sub_epithet = fullname_split[subrank_idx + 1]                                                                                          
          if epithet == sub_epithet:                                                                                                             
              autonym = True         

## 處理 illegitimate & invalid 名稱

In [ ]:
redlistRaw = []
with open('../../workspace/illegitimate.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='|', quotechar='"')
    for row in csvreader:
        redlistRaw.append(row)
il = redlistRaw

In [ ]:
QIL = ''
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(CREATE_TABLE)
        conn.commit()

## 比對 IPNI 學名資料庫

比對學名資料庫，確認作者名稱一致性

In [4]:
import
QUERY = 'SELECT fullname,zh_name FROM twredlist2017;'
with conn:
    with conn.cursor() as curs:                                                                                                                           
        curs.execute(QUERY)
        fullnames = curs.fetchall()
fullnames[0]

In [159]:
import requests
from lxml import html
url = 'http://www.ipni.org/ipni/simplePlantNameSearch.do'
with requests.session() as s:
    s.get(url)
    cookies = s.cookies['JSESSIONID']    
#http://www.ipni.org/ipni/simplePlantNameSearch.do;
# jsessionid=?
# find_wholeName=
# output_format=
# query_type=by_query
# back_page=query_ipni.html

In [171]:
resCont = []
for r in range(0, len(fullnames)):
    simpleName = g.fmtnameNew(fullnames[r][0], doformat=False)[0]
    simpleName = simpleName.replace('×','')
    # check autonyms
    if len(simpleName.split(' ')) == 4:
        snList = simpleName.split(' ')
        if snList[1] == snList[3]:
            simpleName = ' '.join([snList[0], snList[1]])
    # getData
    print([r, simpleName])
    oformat = 'delimited-minimal'
    searchDetail = {'find_wholeName': simpleName, 'output_format': oformat, \
                    'query_type': 'by_query'}
    # cookies
    urlQuery = url + ';' + '='.join(['jsessionid', cookies])
    rg = requests.get(urlQuery, params = searchDetail)
    if 'Id%Version%Family%Full name without family and authors%Authors' in rg.text:
        resCont.append([simpleName, rg.text])
        print(rg.text)

[0, 'Hylodesmum podocarpum']
Id%Version%Family%Full name without family and authors%Authors
1014622-1%1.5%Leguminosae%Hylodesmum podocarpum%(DC.) H.Ohashi & R.R.Mill

[1, 'Ficus ampelos']
Id%Version%Family%Full name without family and authors%Authors
60458494-2%1.1%Moraceae%Ficus ampelos%J.Koenig ex Roxb.
852377-1%1.1.2.1.1.3%Moraceae%Ficus ampelos%Burm.f.
852378-1%1.1.2.2.1.1.2.1%Moraceae%Ficus ampelos%J.Koenig ex Roxb.

[2, 'Dryopteris nidus']
Id%Version%Family%Full name without family and authors%Authors
77123368-1%1.3%Dryopteridaceae%Dryopteris nidus%(Baker) Li Bing Zhang

[3, 'Quercus longinux var. kuoi']
Id%Version%Family%Full name without family and authors%Authors
952067-1%1.5%Fagaceae%Quercus longinux var. kuoi%(Liao) C.F.Shen

[4, 'Adiantum ×meishanianum']
[5, 'Neottia meifongensis']
Id%Version%Family%Full name without family and authors%Authors
77098671-1%1.1.2.1%Orchidaceae%Neottia meifongensis%(H.J.Su & C.Y.Hu) T.C.Hsu & S.W.Chung

[6, 'Pteris semipinnata']
Id%Version%Fami

Id%Version%Family%Full name without family and authors%Authors
516049-1%1.2.1.2%Leguminosae%Psoralea corylifolia%L.

[48, 'Lithocarpus synbalanos']
Id%Version%Family%Full name without family and authors%Authors
359016-1%1.3%Fagaceae%Lithocarpus synbalanos%Chun

[49, 'Huperzia serrata var. longipetiolata']
Id%Version%Family%Full name without family and authors%Authors
17520670-1%1.1.2.1.1.3%Lycopodiaceae%Huperzia serrata f. longipetiolata%(Spring) Ching
20004843-1%1.3%Lycopodiaceae%Huperzia serrata var. longipetiolata%(Spring) H.M.Chang
77067507-1%1.1%Lycopodiaceae%Huperzia serrata var. longipetiolata%(Spring) S.R.Ghosh

[50, 'Pericampylus formosanus']
Id%Version%Family%Full name without family and authors%Authors
581273-1%1.4%Menispermaceae%Pericampylus formosanus%Diels

[51, 'Rubus alnifoliolatus']
Id%Version%Family%Full name without family and authors%Authors
734987-1%1.1.2.2.1.4%Rosaceae%Rubus alnifoliolatus%H.Lév. & Vaniot

[52, 'Themelium curtisii']
Id%Version%Family%Full name wit

Id%Version%Family%Full name without family and authors%Authors
324087-2%1.4%Davalliaceae%Davallia trichomanoides%Blume
17080970-1%1.1.2.1.1.3%Davalliaceae%Davallia trichomanoides%Blume
17080980-1%1.1.2.1.1.1%Davalliaceae%Davallia trichomanoides%Hook.
17080990-1%1.2.2.2.1.1%Davalliaceae%Davallia trichomanoides%Bedd.
17319190-1%1.4%Davalliaceae%Davallia trichomanoides%Blume

[96, 'Lepisorus miyoshianus']
Id%Version%Family%Full name without family and authors%Authors
77100106-1%1.1%Polypodiaceae%Lepisorus miyoshianus%(Makino) Fraser-Jenk. & Subh.Chandra

[97, 'Parthenium hysterophorus']
Id%Version%Family%Full name without family and authors%Authors
236510-1%1.2.1.3%Asteraceae%Parthenium hysterophorus%L.
50243-3%1.1%Asteraceae%Parthenium hysterophorus%L.

[98, 'Vicia hirsuta']
Id%Version%Family%Full name without family and authors%Authors
73213-3%1.1%Fabaceae%Vicia hirsuta%(L.) Gray
524809-1%1.1.2.1.1.4%Leguminosae%Vicia hirsuta%Fisch. ex Ser.
524810-1%1.4.2.1%Leguminosae%Vicia hirsuta%(L.

Id%Version%Family%Full name without family and authors%Authors
613360-1%1.3%Onagraceae%Ludwigia hyssopifolia%(G.Don) Exell
104719-3%1.1%Onagraceae%Ludwigia hyssopifolia%(G.Don) Exell

[141, 'Ludwigia perennis']
Id%Version%Family%Full name without family and authors%Authors
613408-1%1.5%Onagraceae%Ludwigia perennis%L.
613409-1%1.4%Onagraceae%Ludwigia perennis%Burm.f.
105029-3%1.1%Onagraceae%Ludwigia perennis%L.

[142, 'Benthamidia japonica var. chinensis']
[143, 'Pinus fragilissima']
Id%Version%Family%Full name without family and authors%Authors
70029794-1%1.1.2.1.1.2%Pinaceae%Pinus fragilissima%Businský

[144, 'Kandelia obovata']
Id%Version%Family%Full name without family and authors%Authors
70028704-1%1.2.2.1.2.1%Rhizophoraceae%Kandelia obovata%Sheue, H.Y.Liu & J.W.H.Yong

[145, 'Persicaria conspicua']
Id%Version%Family%Full name without family and authors%Authors
694945-1%1.1.2.1.1.1%Polygonaceae%Persicaria conspicua%Nakai

[146, 'Photinia serratifolia var. lasiopetala']
Id%Version%F

Id%Version%Family%Full name without family and authors%Authors
115162-1%1.3.1.4%Boraginaceae%Cynoglossum lanceolatum%B.Heyne ex Wall.
115164-1%1.1.2.1.1.2%Boraginaceae%Cynoglossum lanceolatum%Forssk.
115165-1%1.1.2.1.1.4.2.1.1.1%Boraginaceae%Cynoglossum lanceolatum%Hochst. ex DC.

[185, 'Rosa taiwanensis']
Id%Version%Family%Full name without family and authors%Authors
734465-1%1.1.2.1.1.1.2.1.1.1%Rosaceae%Rosa taiwanensis%Nakai

[186, 'Smithia ciliata']
Id%Version%Family%Full name without family and authors%Authors
518643-1%1.4%Leguminosae%Smithia ciliata%Royle

[187, 'Radiogrammitis subnervosa']
Id%Version%Family%Full name without family and authors%Authors
77162891-1%1.2%Grammitidaceae%Radiogrammitis subnervosa%T.C.Hsu

[188, 'Ipomoea carnea subsp. fistulosa']
Id%Version%Family%Full name without family and authors%Authors
878619-1%1.7%Convolvulaceae%Ipomoea carnea subsp. fistulosa%(Mart. ex Choisy) D.F.Austin
1115630-2%1.3%Convolvulaceae%Ipomoea carnea subsp. fistulosa%(Choisy) D.F.A

...

Id%Version%Family%Full name without family and authors%Authors
296199-1%1.1.2.1.1.1.2.1.1.2.2.1%Fagaceae%Quercus glauca%Martrin-Donos & Timb.-Lagr.
927477-1%1.6%Fagaceae%Quercus glauca%Thunb.
296196-1%1.1.2.3.1.4%Fagaceae%Quercus glauca%Bürger ex Blume
296197-1%1.1.2.1.1.2%Fagaceae%Quercus glauca%Bosc
296198-1%1.1.2.1.1.2.2.1%Fagaceae%Quercus glauca%Thunb.

[4965, 'Zingiber kawagoii']
Id%Version%Family%Full name without family and authors%Authors
873051-1%1.3%Zingiberaceae%Zingiber kawagoii%Hayata

[4966, 'Galium spurium']
Id%Version%Family%Full name without family and authors%Authors
750665-1%1.5%Rubiaceae%Galium spurium%L.
750666-1%1.2%Rubiaceae%Galium spurium%Spreng.
107205-2%1.2%Rubiaceae%Galium spurium%L.
107206-2%1.2%Rubiaceae%Galium spurium%L.
750663-1%1.1.2.1.1.1%Rubiaceae%Galium spurium%All.
750664-1%1.2.1.2%Rubiaceae%Galium spurium%Huds.

[4967, 'Salix morii']
Id%Version%Family%Full name without family and authors%Authors
778222-1%1.4%Salicaceae%Salix morii%Hayata

[4968, 'Am

Id%Version%Family%Full name without family and authors%Authors
755044-1%1.3%Rubiaceae%Lasianthus microphyllus%Elmer

[5010, 'Euphorbia humifusa']
Id%Version%Family%Full name without family and authors%Authors
346840-1%1.1.2.1.1.1%Euphorbiaceae%Euphorbia humifusa%Willd.
346841-1%1.1.2.1.1.5.2.1%Euphorbiaceae%Euphorbia humifusa%hort. ex Boiss.

[5011, 'Rhaphidophora liukiuensis']
Id%Version%Family%Full name without family and authors%Authors
88525-1%1.4.2.1.1.1%Araceae%Rhaphidophora liukiuensis%Hatus.

[5012, 'Pinanga tashiroi']
Id%Version%Family%Full name without family and authors%Authors
669161-1%1.3%Arecaceae%Pinanga tashiroi%Hayata

[5013, 'Rhachidosorus pulcher']
Id%Version%Family%Full name without family and authors%Authors
17453600-1%1.5%Woodsiaceae%Rhachidosorus pulcher%(Tagawa) Ching

[5014, 'Myriophyllum aquaticum']
Id%Version%Family%Full name without family and authors%Authors
430396-1%1.2.1.5%Haloragaceae%Myriophyllum aquaticum%(Vell.) Verdc.
153047-3%1.1%Haloragaceae%Myriop

Id%Version%Family%Full name without family and authors%Authors
17119990-1%1.6%Dennstaedtiaceae%Histiopteris incisa%(Thunb.) J.Sm.
129732-3%1.1%Dennstaedtiaceae%Histiopteris incisa%(Thunb.) J.Sm.

[5055, 'Hypolepis alpina']
Id%Version%Family%Full name without family and authors%Authors
17126200-1%1.5%Dennstaedtiaceae%Hypolepis alpina%(Blume) Hook.

[5056, 'Radiogrammitis beddomeana']
Id%Version%Family%Full name without family and authors%Authors
77085653-1%1.1%Grammitidaceae%Radiogrammitis beddomeana%(Alderw.) Parris

[5057, 'Aeschynomene americana var. glandulosa']
Id%Version%Family%Full name without family and authors%Authors
5895-2%1.4%Fabaceae%Aeschynomene americana var. glandulosa%(Poir.) Rudd

[5058, 'Vernicia fordii']
Id%Version%Family%Full name without family and authors%Authors
358198-1%1.2.1.3.2.1.1.3%Euphorbiaceae%Vernicia fordii%(Hemsl.) Airy Shaw
161343-3%1.1.2.1%Euphorbiaceae%Vernicia fordii%Airy Shaw

[5059, 'Kummerowia striata']
Id%Version%Family%Full name without family

Id%Version%Family%Full name without family and authors%Authors
175003-1%1.6%Asteraceae%Adenostemma lavenia%(L.) Kuntze
90888-3%1.1%Asteraceae%Adenostemma lavenia%(L.) Kuntze

[5101, 'Salix taiwanalpina']
Id%Version%Family%Full name without family and authors%Authors
778985-1%1.2%Salicaceae%Salix taiwanalpina%Kimura

[5102, 'Gymnocarpium oyamense']
Id%Version%Family%Full name without family and authors%Authors
17332560-1%1.5%Woodsiaceae%Gymnocarpium oyamense%Ching

[5103, 'Lobelia nummularia']
Id%Version%Family%Full name without family and authors%Authors
143608-1%1.4%Campanulaceae%Lobelia nummularia%Lam.

[5104, 'Silene morrisonmontana']
Id%Version%Family%Full name without family and authors%Authors
157861-1%1.1.2.2.1.3%Caryophyllaceae%Silene morrisonmontana%(Hayata) Ohwi & H.Ohashi

[5105, 'Gnaphalium luteoalbum']
Id%Version%Family%Full name without family and authors%Authors
209666-1%1.2.1.1.2.1.1.1%Asteraceae%Gnaphalium luteoalbum%L.
111195-2%1.2.2.1.1.3%Asteraceae%Gnaphalium luteoa

Id%Version%Family%Full name without family and authors%Authors
407074-1%1.3%Poaceae%Leptochloa chinensis%Nees
407073-1%1.5%Poaceae%Leptochloa chinensis%Nees
393372-1%1.1.1.4%Poaceae%Leptochloa chinensis%Nees
61317-3%1.1%Poaceae%Leptochloa chinensis%(Roth) Nees

[5146, 'Leptochloa panicea']
Id%Version%Family%Full name without family and authors%Authors
407138-1%1.6%Poaceae%Leptochloa panicea%(Retz.) Ohwi

[5147, 'Paspalum fimbriatum']
Id%Version%Family%Full name without family and authors%Authors
414155-1%1.4%Poaceae%Paspalum fimbriatum%Kunth

[5148, 'Setaria parviflora']
Id%Version%Family%Full name without family and authors%Authors
943811-1%1.1.2.2.1.2%Poaceae%Setaria parviflora%(Poir.) Kerguélen
1182837-2%1.3%Poaceae%Setaria parviflora%(Poir.) Kerguélen

[5149, 'Setaria palmifolia']
Id%Version%Family%Full name without family and authors%Authors
421711-1%1.1.2.1.1.3%Poaceae%Setaria palmifolia%Stapf
61557-3%1.1.2.1%Poaceae%Setaria palmifolia%Stapf

[5150, 'Egeria densa']
Id%Version%Fam

In [174]:
resCont = []
for r in range(0, len(fullnames)):
    simpleName = g.fmtnameNew(fullnames[r][0], doformat=False)[0]
    if '×' in simpleName:
        simpleName = simpleName.replace('×','')
        # check autonyms
        if len(simpleName.split(' ')) == 4:
            snList = simpleName.split(' ')
            if snList[1] == snList[3]:
                simpleName = ' '.join([snList[0], snList[1]])
        # getData
        print([r, simpleName])
        oformat = 'delimited-minimal'
        searchDetail = {'find_wholeName': simpleName, 'output_format': oformat, \
                        'query_type': 'by_query'}
        # cookies
        urlQuery = url + ';' + '='.join(['jsessionid', cookies])
        rg = requests.get(urlQuery, params = searchDetail)
        if 'Id%Version%Family%Full name without family and authors%Authors' in rg.text:
            resCont.append([simpleName, rg.text])
            print(rg.text)

[4, 'Adiantum meishanianum']
Id%Version%Family%Full name without family and authors%Authors
77100319-1%1.3%Adiantaceae%Adiantum meishanianum%F.S.Hsu ex Yea C.Liu & W.L.Chiou

[259, 'Calanthe hsinchuensis']
Id%Version%Family%Full name without family and authors%Authors
77142919-1%1.2%Orchidaceae%Calanthe × hsinchuensis%Y.I Lee

[270, 'Epidendrum obrienianum']
Id%Version%Family%Full name without family and authors%Authors
113958-3%1.1%Orchidaceae%Epidendrum obrienianum%Rolfe
632357-1%1.1.2.1.1.5%Orchidaceae%Epidendrum × obrienianum%Rolfe

[396, 'Begonia chungii']
Id%Version%Family%Full name without family and authors%Authors
77099695-1%1.1%Begoniaceae%Begonia × chungii%C.I Peng & S.M.Ku

[503, 'Athyrium purpurascens']
Id%Version%Family%Full name without family and authors%Authors
17430370-1%1.5%Woodsiaceae%Athyrium × purpurascens%(Tagawa) Sa.Kurata

[523, 'Dryopteris subreflexipinna']
Id%Version%Family%Full name without family and authors%Authors
17392680-1%1.3%Dryopteridaceae%Dryopteris

In [ ]:
http://taibnet.sinica.edu.tw/chi/taibnet_species_list.php?T2=蓮實藤&T2_new_value=true&fr=y